## Домашняя работа. Uplift-моделирование

### Задания

1. Скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention \

2. Там поле `conversion` - это целевая переменная, а `offer` - коммуникация. Переименовать поля (`conversion` -> `target`, `offer` -> `treatment`) и привести поле `treatment` к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

3. Cделать разбиение набора данных не тренировочную и тестовую выборки

4. Cделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

5. Провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

6. В конце вывести единую таблицу сравнения метрик `uplift@10%`, `uplift@20%` этих 3 моделей

7. Построить модель `UpliftTreeClassifier` и попытаться описать словами полученное дерево

8. (Опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле `conversion`) от значения `uplift`:

    - Cделать прогноз и получить `uplift` для тестовой выборки 
    - Отсортировать тестовую выборку по uplift по убыванию 
    - Разбить на децили (`pandas qcut` вам в помощь) 
    - Для каждого дециля посчитать среднюю `conversion`

9. (Опционально) построить модель `UpliftRandomForestClassifier` и попытаться описать словами полученное дерево

**Ссылки:**
- https://towardsdatascience.com/a-quick-uplift-modeling-introduction-6e14de32bfe0
- https://habr.com/ru/company/ru_mts/blog/485980/#reference1
- https://en.wikipedia.org/wiki/Uplift_modelling
- https://www.youtube.com/watch?v=yFQAIJBYXI0
- https://www.youtube.com/watch?v=jCUcYiBK03I
- https://www.uplift-modeling.com/en/latest/
- https://arxiv.org/pdf/1809.04559.pdf
- https://catboost.ai/docs/concepts/about.html

**Библиотеки и пакеты:**
- `causalml`
- `sklift`
- `catboost`

---

In [ ]:
# Загрузка необходимых библиотек

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from pprint import pprint

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from sklift.models import TwoModels
from sklift.models import ClassTransformation

from catboost import CatBoostClassifier

from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

In [ ]:
# Загрузка данных

df_customer = pd.read_csv('HW_data.csv')

# Проверка

df_customer.head(10)

---

In [ ]:
# Переименование полей

df_customer = df_customer.rename(columns = {'conversion': 'target', 'offer': 'treatment'})

# Бинарные значения

df_customer['treatment'] = df_customer['treatment'].apply(lambda i : 0 if i == 'No Offer' else 1)

In [ ]:
# Train test split

X_train, X_test, y_train, y_test = train_test_split(df_customer.drop('target', axis = 1), df_customer['target'], test_size = 0.33, random_state = 42)

In [ ]:
# One-Hot Encoding (Dummies)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
# Новые переменные

treat_train = X_train['treatment']
treat_test = X_test['treatment']

models_results = {
    'approach': [],
    'uplift@10%': [],
    'uplift@30%': []
}

In [ ]:
%%time

# Модель с признаком коммуникации

sm = SoloModel(CatBoostClassifier(iterations = 100, thread_count = 2, random_state = 42, silent = True))
sm = sm.fit(X_train, y_train, treat_train)

uplift_sm = sm.predict(X_test)

sm_score_10 = uplift_at_k(y_true = y_test, uplift = uplift_sm, treatment = treat_test, strategy = 'by_group', k = 0.1)
sm_score_30 = uplift_at_k(y_true = y_test, uplift = uplift_sm, treatment = treat_test, strategy = 'by_group', k = 0.3)

models_results['approach'].append('SoloModel')
models_results['uplift@10%'].append(sm_score_10)
models_results['uplift@30%'].append(sm_score_30)

In [ ]:
# Вероятность при взаимодействии

sm_trmnt_preds = sm.trmnt_preds_

# Вероятность без взаимодействия

sm_ctrl_preds = sm.ctrl_preds_

# Построим графики

plot_uplift_preds(trmnt_preds = sm_trmnt_preds, ctrl_preds = sm_ctrl_preds)

In [ ]:
%%time

# Модель с трансформацией target

ct = ClassTransformation(CatBoostClassifier(iterations = 100, thread_count = 2, random_state = 42, silent = True))
ct = ct.fit(X_train, y_train, treat_train)

uplift_ct = ct.predict(X_test)

ct_score_10 = uplift_at_k(y_true = y_test, uplift = uplift_ct, treatment = treat_test, strategy = 'by_group', k = 0.1)
ct_score_30 = uplift_at_k(y_true = y_test, uplift = uplift_ct, treatment = treat_test, strategy = 'by_group', k = 0.3)

models_results['approach'].append('ClassTransformation')
models_results['uplift@10%'].append(ct_score_10)
models_results['uplift@30%'].append(ct_score_30)

In [ ]:
%%time

# Две независимые модели

tm = TwoModels(
    estimator_trmnt = CatBoostClassifier(iterations = 100, thread_count = 2, random_state = 42, silent = True), 
    estimator_ctrl = CatBoostClassifier(iterations = 100, thread_count = 2, random_state = 42, silent = True), 
    method = 'vanilla')

tm = tm.fit(
    X_train, y_train, treat_train)

uplift_tm = tm.predict(X_test)

tm_score_10 = uplift_at_k(y_true = y_test, uplift = uplift_tm, treatment = treat_test, strategy = 'by_group', k = 0.1)
tm_score_30 = uplift_at_k(y_true = y_test, uplift = uplift_tm, treatment = treat_test, strategy = 'by_group', k = 0.3)

models_results['approach'].append('TwoModels')
models_results['uplift@10%'].append(tm_score_10)
models_results['uplift@30%'].append(tm_score_30)

In [ ]:
# Отобразим результаты в Dataframe

df_result = pd.DataFrame(data = models_results).sort_values('uplift@30%', ascending = False).set_index('approach')

df_result

---

### UpliftTreeClassifier

In [ ]:
X_train_tree = X_train.copy()

features = [c for c in X_train_tree]

In [ ]:
# Модель

uplift_model = UpliftTreeClassifier(max_depth = 4,
                                    min_samples_leaf = 200,
                                    min_samples_treatment = 50,
                                    n_reg = 100,
                                    evaluationFunction = 'KL',
                                    control_name = 'control')

uplift_model.fit(X_train_tree.values,
                 treatment = treat_train.map({
                     1: 'treatment1',
                     0: 'control'
                 }).values,
                 y = y_train)

graph = uplift_tree_plot(uplift_model.fitted_uplift_tree, features)
Image(graph.create_png())